[View in Colaboratory](https://colab.research.google.com/github/alamiin/VGG16/blob/master/VGG16_networks_with_data_Generator.ipynb)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import random
import os
import glob
import cv2
import datetime
import pandas as pd
import time
import h5py
import csv
from scipy import misc, ndimage
from scipy.misc import imresize, imsave

from sklearn.cross_validation import KFold, train_test_split
from sklearn.metrics import log_loss, confusion_matrix
from sklearn.utils import shuffle

from PIL import Image, ImageChops, ImageOps

import matplotlib.pyplot as plt
#os.environ['KERAS_BACKEND'] = 'theano'
from keras import backend as K
from keras.callbacks import EarlyStopping, Callback
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras import optimizers
from keras.models import Sequential, model_from_json
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, Activation, Dropout, Flatten, Dense

%matplotlib inline


/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [0]:
train_path =  "/content/drive/My Drive/plamprint/database/database/"
weights_path = "/content/drive/My Drive/Colab Notebooks/weight/vgg16_weights.h5"
bottleneck_model_weights_path = "/content/drive/My Drive/Colab Notebooks/weight/bottleneck_weights.h5"


batch_size = 16
nb_epoch = 50
bottleneck_epoch = 3  
val_split = .10  

num_classes = 168
img_width, img_height = 224, 224
num_channels = 3


In [0]:
gen = ImageDataGenerator( 
    rotation_range=20.,
    fill_mode='constant',
    data_format="channels_last",
)



In [0]:
def get_class_number(j):
    
    if(len(j.split(' '))==2):
        return int(j.split(' ')[0].split('_')[1])
    
    else:
        return int(j.split('(')[0].split('_')[1])

In [0]:
def load_COEP():
    X_train = []
    X_train_id = []
    y_train = []
    start_time = time.time()

    print('Loading training images...')
    
    
    for _, dirnames, filenames in os.walk(train_path):
        all_dir.append(dirnames)


    classs = []
    label = []
    
    
    i = 0
    for _, dirnames, filenames in os.walk(train_path):
        print("Totala number of Instance of COEP dataset: {}".format(len(filenames)))
        for j in filenames:
            image = cv2.imread(dirt+j)
            
            roi=image[0:1200, 400:1600] 
            img_rotation = cv2.resize(roi, (img_width, img_height ))
            hsv = cv2.cvtColor(img_rotation, cv2.COLOR_BGR2HSV)
            lower_skin = np.array([0,20,70], dtype=np.uint8)
            upper_skin = np.array([20,255,255], dtype=np.uint8)
            mask = cv2.inRange(hsv, lower_skin, upper_skin)
            image = img_rotation.copy()
            image[np.where(mask==0)] = 0
            
           
            image = np.expand_dims(image,0)
            aug_iter = gen.flow(image)
            aug_images = [next(aug_iter)[0].astype(np.uint8) for i in range(4)]
            for img in aug_images:
             
              X_train.append(img)
              label.append(get_class_number(j))
            if i % 100 == 0:
              print("{} Number image have read complete. Next image is reading".format(i))
            i += 1


    
    print('Training data load time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_train, label, classs


In [0]:
def normalize_train_data():
    train_data, train_target, train_id = load_COEP()
   
    train_data = np.array(train_data, dtype=np.uint8)
    train_target = np.array(train_target, dtype=np.uint8)
    

    train_data = train_data.astype('float32')
    train_data = train_data / 255
    train_target = np_utils.to_categorical(train_target, 168)

    print('Shape of training data:', train_data.shape)
    return train_data, train_target, train_id

In [8]:
train_data, train_target, train_id = normalize_train_data()

Loading training images...
Totala number of Instance of COEP dataset: 1303
0 Number image have read complete. Next image is reading
100 Number image have read complete. Next image is reading
200 Number image have read complete. Next image is reading
300 Number image have read complete. Next image is reading
400 Number image have read complete. Next image is reading
500 Number image have read complete. Next image is reading
600 Number image have read complete. Next image is reading
700 Number image have read complete. Next image is reading
800 Number image have read complete. Next image is reading
900 Number image have read complete. Next image is reading
1000 Number image have read complete. Next image is reading
1100 Number image have read complete. Next image is reading
1200 Number image have read complete. Next image is reading
1300 Number image have read complete. Next image is reading
Training data load time: 108.82 seconds
Shape of training data: (5212, 250, 250, 3)


In [0]:
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1./255)

    # build the VGG16 network
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=( img_width, img_height, 3)))

    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # load the weights of the VGG16 networks
    f = h5py.File(weights_path)
    for k in range(f.attrs['nb_layers']):
        if k >= len(model.layers):
            # we don't look at the last (fully-connected) layers in the savefile
            break
        g = f['layer_{}'.format(k)]
        weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
        if np.array(weights).shape[0] > 0:
            weights[0] = np.transpose(np.array(weights[0])[:, :, ::-1, ::-1], (2, 3, 1, 0))
        model.layers[k].set_weights(weights)
    f.close()
    print('Model loaded.')
    
    # create validation split
    X_train, X_valid, Y_train, Y_valid = train_test_split(train_data, train_target, test_size=val_split)

    # create generator for train data
    generator = datagen.flow(
            X_train,
            Y_train,
            batch_size=batch_size,
            shuffle=False)
    print("Start pridict generator...")
    # save train features to .npy file
    bottleneck_features_train = model.predict_generator(generator)
    np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)
    print("shape of bottleneck_features_train {}".format(bottleneck_features_train.shape))
    # create generator for validation data
    generator = datagen.flow(
            X_valid,
            Y_valid,
            batch_size=batch_size,
            shuffle=False)
    
    # save validation features to .npy file
    bottleneck_features_validation = model.predict_generator(generator )
    np.save(open('bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)
    print("shape of bottleneck_features_validation {}".format(bottleneck_features_validation.shape))
    return Y_train, Y_valid

In [0]:
def train_bottleneck_model():
    train_labels, validation_labels = save_bottleneck_features()

    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    
    print(train_data.shape)
    print(train_labels.shape)
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

    model.fit(train_data,
              train_labels,
              nb_epoch=5,
              #batch_size=batch_size,
              #validation_data=(validation_data, validation_labels),
              #callbacks=[early_stopping],
              #verbose=2
             )
    
    model.save_weights(bottleneck_model_weights_path)
    return model

In [0]:
train_bottleneck_model() 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_1")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_2")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv2_1")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv2_2")`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu", name="conv3_1")`
/usr/local

Model loaded.


In [0]:
def build_model():
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=( img_width, img_height, 3)))

    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    # load the weights of the VGG16 networks
    f = h5py.File(weights_path)
    for k in range(f.attrs['nb_layers']):
        if k >= len(model.layers):
            # we don't look at the last (fully-connected) layers in the savefile
            break
        g = f['layer_{}'.format(k)]
        weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
        if np.array(weights).shape[0] > 0:
            weights[0] = np.transpose(np.array(weights[0])[:, :, ::-1, ::-1], (2, 3, 1, 0))
        model.layers[k].set_weights(weights)
    f.close()
    
    # build a classifier model to put on top of the convolutional model
    bottleneck_model = Sequential()
    bottleneck_model.add(Flatten(input_shape=model.output_shape[1:]))
    bottleneck_model.add(Dense(256, activation='relu'))
    bottleneck_model.add(Dropout(0.5))
    bottleneck_model.add(Dense(num_classes, activation='softmax'))
    
    # load weights from bottleneck model
    bottleneck_model.load_weights(bottleneck_model_weights_path)

    # add the model on top of the convolutional base
    model.add(bottleneck_model)

    # set the first 25 layers (up to the last conv block)
    # to non-trainable (weights will not be updated)
    for layer in model.layers[:25]:
        layer.trainable = False
        
    # compile the model with a SGD/momentum optimizer
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=['accuracy'])
    return model 

In [0]:

X_train, X_valid, Y_train, Y_valid = train_test_split(train_data, train_target, test_size=val_split)
model = build_model()
history = model.fit(X_train,
          Y_train,
          nb_epoch=200,
         # shuffle=True,
          #verbose=1,
          validation_data=(X_valid, Y_valid),
          #callbacks=callbacks
         )


In [0]:
print(history.history.keys())
#  "Accuracy"
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()